# 4-2 Load Data

## Slicing 1D Array

In [1]:
nums = [0, 1, 2, 3, 4]

In [2]:
print(nums)

[0, 1, 2, 3, 4]


In [3]:
#index 2에서 4전까지 가져와라. (앞 포함, 뒤 비포함) -> 주의!
print(nums[2:4])

[2, 3]


In [4]:
print(nums[2:])    #어디까지인지 명시 X (2이후로 다 가져옴)

[2, 3, 4]


In [5]:
print(nums[:2])    #어디서부터인지 명시 X(2이전까지 다 가져옴)

[0, 1]


In [6]:
print(nums[:])    #다 가져와

[0, 1, 2, 3, 4]


In [7]:
print(nums[:-1])    #- 붙으면 뒤에서 부터. -> 마지막 index전까지 가져와라

[0, 1, 2, 3]


In [8]:
nums[2:4] = [8, 9]

In [9]:
print(nums)

[0, 1, 8, 9, 4]


## Slicing 2D Array

In [10]:
import numpy as np

In [11]:
#3 by 4 tensor
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

In [12]:
print(b)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [13]:
b[:, 1]

array([ 2,  6, 10])

In [14]:
b[-1]

array([ 9, 10, 11, 12])

In [15]:
b[-1, :]

array([ 9, 10, 11, 12])

In [16]:
b[-1, ...]

array([ 9, 10, 11, 12])

In [17]:
b[0:2, :]

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

#### 어디 차원이 어떻게 slicing 되는지 설명할 수 있어야 함.

## Loading Data from .csv file

In [6]:
import numpy as np

In [7]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)

In [10]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [11]:
print(x_data.shape) # x_data shape
print(len(x_data))  # x_data 길이
print(x_data[:5])   # 첫 다섯 개

(25, 3)
25
[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]]


In [12]:
print(y_data.shape) # y_data shape
print(len(y_data))  # y_data 길이
print(y_data[:5])   # 첫 다섯 개

(25, 1)
25
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]]


## Imports

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [15]:
# For reproducibility
torch.manual_seed(1)

## Low-level Implementation

In [16]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 26811.960938
Epoch    1/20 Cost: 9920.530273
Epoch    2/20 Cost: 3675.298828
Epoch    3/20 Cost: 1366.260498
Epoch    4/20 Cost: 512.542480
Epoch    5/20 Cost: 196.896667
Epoch    6/20 Cost: 80.190987
Epoch    7/20 Cost: 37.038689
Epoch    8/20 Cost: 21.081348
Epoch    9/20 Cost: 15.178763
Epoch   10/20 Cost: 12.993682
Epoch   11/20 Cost: 12.183030
Epoch   12/20 Cost: 11.880535
Epoch   13/20 Cost: 11.765960
Epoch   14/20 Cost: 11.720860
Epoch   15/20 Cost: 11.701432
Epoch   16/20 Cost: 11.691512
Epoch   17/20 Cost: 11.685116
Epoch   18/20 Cost: 11.680002
Epoch   19/20 Cost: 11.675382
Epoch   20/20 Cost: 11.670952


## High-level Implementation with nn.Module

In [17]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [18]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 28693.490234
Epoch    1/20 Cost: 10618.750000
Epoch    2/20 Cost: 3936.015381
Epoch    3/20 Cost: 1465.219482
Epoch    4/20 Cost: 551.693848
Epoch    5/20 Cost: 213.934692
Epoch    6/20 Cost: 89.052246
Epoch    7/20 Cost: 42.875988
Epoch    8/20 Cost: 25.799639
Epoch    9/20 Cost: 19.482420
Epoch   10/20 Cost: 17.143108
Epoch   11/20 Cost: 16.274498
Epoch   12/20 Cost: 15.949728
Epoch   13/20 Cost: 15.825985
Epoch   14/20 Cost: 15.776577
Epoch   15/20 Cost: 15.754650
Epoch   16/20 Cost: 15.742910
Epoch   17/20 Cost: 15.734902
Epoch   18/20 Cost: 15.728330
Epoch   19/20 Cost: 15.722221
Epoch   20/20 Cost: 15.716338


## Dataset and DataLoader

### 너무 데이터가 크면 x_data, y_data 를 전부 다 가져오지 말고, 필요한 배치만 가져올 수 밖에 없다.